In [176]:
!pip install konlpy
!pip install torchtext

In [177]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
from konlpy.tag import Okt
#주요 참고 PyTorch로 시작하는 딥 러닝 입문, 유원준
from torchtext import data  
from konlpy.tag import Mecab
import urllib.request
import pandas as pd
import random

In [178]:

from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [179]:
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

In [180]:
'''
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")
'''

'\nurllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")\nurllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")\n'

In [181]:
'''
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')
train_df.head(10)
'''

"\ntrain_df = pd.read_table('ratings_train.txt')\ntest_df = pd.read_table('ratings_test.txt')\ntrain_df.head(10)\n"

In [182]:
'''
train_df.drop_duplicates(subset=['document'], inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df.to_csv("/content/gdrive/My Drive/datas/ratings_train.csv", mode='w',index=False)
test_df.to_csv("/content/gdrive/My Drive/datas/ratings_test.csv", mode='w',index=False)
'''

'\ntrain_df.drop_duplicates(subset=[\'document\'], inplace=True)\ntest_df.drop_duplicates(subset=[\'document\'], inplace=True)\ntrain_df.dropna(inplace=True)\ntest_df.dropna(inplace=True)\n\ntrain_df.to_csv("/content/gdrive/My Drive/datas/ratings_train.csv", mode=\'w\',index=False)\ntest_df.to_csv("/content/gdrive/My Drive/datas/ratings_test.csv", mode=\'w\',index=False)\n'

In [183]:
tokenizer = Okt()
# 필드 정의
ID = data.Field(sequential = False,
                use_vocab = False) 

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs, 
                  lower=True,
                  batch_first=True,
                  fix_length=256)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)

In [184]:
from torchtext.data import TabularDataset
train_data = TabularDataset(path='/content/gdrive/My Drive/datas/ratings_train.csv', format='csv',fields=[('id', ID), ('document', TEXT), ('label', LABEL)], skip_header=True)
test_data = TabularDataset(path='/content/gdrive/My Drive/datas/ratings_test.csv', format='csv',fields=[('id', ID), ('document', TEXT), ('label', LABEL)], skip_header=True)

In [185]:
class grumodel(nn.Module):
    def __init__(self, embed_dim, vocab_size, hidden_dim, num_layers, batch_size, dropout):
        super(grumodel, self).__init__()
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.relu = nn.ReLU()
        self.batch_size = batch_size
        self.sigmoid = nn.Sigmoid()


        self.norm = nn.BatchNorm1d(self.batch_size)
        self.embed = nn.Embedding(self.vocab_size,self.embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.gru = nn.GRU(self.embed_dim, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)

        self.mlp1 = nn.Linear(self.hidden_dim*4,self.hidden_dim)
        self.mlp2 = nn.Linear(self.hidden_dim,self.hidden_dim//4)
        self.mlp3 = nn.Linear(self.hidden_dim//4,1)

    def forward(self,x):
        x = self.embed(x)
        x = self.dropout(x)

        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        x = torch.cat((x[:,0,:],x[:,-1,:]),dim=-1)

        x = self.dropout(x)
        x = self.mlp1(x)
        x = self.relu(x)
        x = self.mlp2(x)
        x = self.relu(x)
        x = self.mlp3(x)
        x = self.sigmoid(x).squeeze()
        return x


In [195]:
def train(model, optimizer, loss_function,train_iter,DEVICE):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.document.to(DEVICE), batch.label.to(DEVICE)
        optimizer.zero_grad()

        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        loss.backward()
        optimizer.step()

In [196]:
def getF1(y_pred,y,threshold=0.5):
    
    yp = [1 if x > threshold else 0 for x in y_pred]

    pp = 0
    pf = 0
    fp = 0
    ff = 0
    for i in range(len(y)):
        if y[i] > threshold:
            if yp[i] > threshold: pp += 1
            else: pf += 1
        else:
            if yp[i] < threshold: ff += 1
            else: fp += 1

    precision = pp / (pp + fp + 1e-5) 
    recall = pp / (pp + ff + 1e-5)
    F1 = 2 * precision * recall / (precision + recall + 1e-5)
    acc = (pp + ff) / (len(y) + 1e-5)
    return F1, acc

In [197]:
def evaluate(model, val_iter, loss_function, DEVICE, batch_size):
    """evaluate model"""
    model.eval()
    total_loss = 0
    total_f1 = 0
    total_acc = 0
    for batch in val_iter:
        x, y = batch.document.to(DEVICE), batch.label.to(DEVICE)
        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        f1, acc = getF1(y_pred,y)
        total_f1 += f1
        total_acc += acc
        total_loss += loss.item()

    size = len(val_iter.dataset) / batch_size
    avg_loss = total_loss / size
    avg_f1 = total_f1 / size
    avg_acc = total_acc / size
    return avg_loss, avg_f1, avg_acc

In [198]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 146182
테스트 샘플의 개수 : 49157


In [199]:
print(vars(train_data[0]))

{'id': '9976970', 'document': ['아', '더빙', '..', '진짜', '짜증나네요', '목소리'], 'label': '0'}


In [200]:
device = torch.device("cuda")
vocab_size = 6000
batch_size = 100
embed_dim = 128
hidden_dim = 256
dropout = 0.6
layers = 1

model = grumodel(embed_dim,vocab_size+2,hidden_dim,layers,batch_size,dropout)
model.to(device)
loss = nn.BCELoss()
lr = 0.002


EPOCHS = 20
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [201]:
TEXT.build_vocab(train_data, min_freq=10, max_size=vocab_size)
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 6002


In [202]:

'''
train_iter, val_iter = data.BucketIterator.splits(
        (train_data, test_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)
'''
from torchtext.data import Iterator
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
val_loader = Iterator(dataset=test_data, batch_size = batch_size)

In [203]:
best_val_f1 = 0
for e in range(1, EPOCHS+1):
    train(model, optimizer, loss,train_loader,device)
    val_loss,val_f1,val_acc = evaluate(model, val_loader, loss, device,batch_size)

    print("[Epoch: %d] val loss : %1.5f    val acc :%4.3f    F1 :%4.3f" % (e, val_loss, val_acc,val_f1))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_f1 or val_f1 > best_val_f1:
        print("Best saved")
        torch.save(model.state_dict(), '/content/gdrive/My Drive/GRUmodel/NLP_esemble_model.pt')
        best_val_f1 = val_f1

[Epoch: 1] val loss : 0.38204    val acc :0.828    F1 :0.613
Best saved
[Epoch: 2] val loss : 0.35676    val acc :0.839    F1 :0.616
Best saved
[Epoch: 3] val loss : 0.34753    val acc :0.847    F1 :0.622
Best saved
[Epoch: 4] val loss : 0.33957    val acc :0.849    F1 :0.630
Best saved
[Epoch: 5] val loss : 0.33959    val acc :0.849    F1 :0.629
[Epoch: 6] val loss : 0.34386    val acc :0.849    F1 :0.631
Best saved
[Epoch: 7] val loss : 0.33919    val acc :0.849    F1 :0.625
[Epoch: 8] val loss : 0.34292    val acc :0.850    F1 :0.621
[Epoch: 9] val loss : 0.34059    val acc :0.849    F1 :0.634
Best saved
[Epoch: 10] val loss : 0.34776    val acc :0.851    F1 :0.626
[Epoch: 11] val loss : 0.34296    val acc :0.850    F1 :0.626
[Epoch: 12] val loss : 0.33639    val acc :0.850    F1 :0.628
[Epoch: 13] val loss : 0.35115    val acc :0.849    F1 :0.623
[Epoch: 14] val loss : 0.34559    val acc :0.848    F1 :0.631
[Epoch: 15] val loss : 0.33965    val acc :0.850    F1 :0.629
[Epoch: 16] v